In [312]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [313]:
pair="CAD_CHF"
granularity = "H1"
ma_list = [16,64] 
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [314]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time','volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [315]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()
for ma in ma_list: 
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True,inplace=True)

In [316]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-10-14T19:00:00.000000000Z,0.72389,0.72414,0.72348,0.72382,0.725890,0.724275
1,2022-10-14T20:00:00.000000000Z,0.72380,0.72469,0.72362,0.72448,0.725667,0.724316
2,2022-10-16T21:00:00.000000000Z,0.72437,0.72464,0.72393,0.72434,0.725459,0.724349
3,2022-10-16T22:00:00.000000000Z,0.72438,0.72465,0.72386,0.72438,0.725314,0.724399
4,2022-10-16T23:00:00.000000000Z,0.72434,0.72474,0.72401,0.72439,0.725199,0.724473


In [317]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [318]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [319]:
df_trades.head()


,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
9,2022-10-17T04:00:00.000000000Z,0.72548,0.72574,0.72538,0.72566,0.724710,0.724753,-0.000043,0.000133,-1
16,2022-10-17T11:00:00.000000000Z,0.72558,0.72640,0.72478,0.72597,0.725014,0.724990,0.000025,-0.000045,1
31,2022-10-18T02:00:00.000000000Z,0.72492,0.72516,0.72426,0.72494,0.725782,0.725785,-0.000002,0.000062,-1
65,2022-10-19T12:00:00.000000000Z,0.72730,0.72984,0.72705,0.72924,0.725334,0.725075,0.000258,-0.000008,1
136,2022-10-24T11:00:00.000000000Z,0.73078,0.73108,0.72982,0.73066,0.730270,0.730328,-0.000058,0.000018,-1


In [320]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [321]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [327]:
df_all = pd.read_pickle("all_trades.pkl")

In [330]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,88747.000000,8.874700e+04,88747.000000,88747.000000,88747.000000,88747.000000,88747.000000,88747.000000,88747.000000
mean,31.259139,-3.938046e-04,0.000478,-0.003324,2.003944,-6.780500,12.136478,63.061151,50.086504
std,54.743054,2.994970e-02,0.028649,1.000000,78.657699,78.390521,12.789937,65.230485,78.755636
min,0.544740,-8.646146e-01,-0.656297,-1.000000,-812.900000,-522.400000,4.000000,8.000000,1.000000
25%,0.884150,-2.250000e-04,-0.000217,-1.000000,-26.600000,-37.400000,4.000000,16.000000,6.000000
50%,1.329220,-6.250000e-07,0.000001,-1.000000,1.100000,-12.800000,8.000000,32.000000,19.000000
75%,2.001320,2.196875e-04,0.000223,1.000000,30.200000,10.000000,16.000000,96.000000,67.000000
max,174.546000,6.221250e-01,1.131125,1.000000,1327.900000,1327.900000,64.000000,256.000000,1516.000000


In [322]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)
df_trades["DURATION"] = [x.total_seconds() / 3600 for x in df_trades.DURATION]
df_trades[['time','DURATION']].head()

In [ ]:
df_trades["GAIN"].sum()

-5.39999999999978

In [ ]:
df_plot = df_ma.iloc[10:200].copy()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1),opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor='#CC2E3C',
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list: 
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict( width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10), 
    font = dict(size=10,color="#e1e1e1"),
    paper_bgcolor='#1e1e1e', 
    plot_bgcolor='#1e1e1e'      
    )
fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()